# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
#!pip install gmaps 

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
cities_df = pd.read_csv('../output_data/cities.csv', index_col = False).dropna()
cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sakakah,29.9697,40.2064,81.21,16,0,14.09,SA,1633810422
1,Nikolskoye,59.7035,30.7861,53.87,84,100,5.70,RU,1633810343
2,Yellowknife,62.4560,-114.3525,45.41,98,100,1.99,CA,1633810359
3,Kyra,49.5776,111.9753,28.54,81,13,5.41,RU,1633810423
4,Mishan,45.5500,131.8833,59.18,91,100,14.94,CN,1633810423
5,Guerrero Negro,27.9769,-114.0611,76.57,52,5,22.70,MX,1633810307
6,Lagoa,39.0500,-27.9833,69.57,77,100,19.28,PT,1633810291
7,Muros,42.7762,-9.0603,66.24,69,88,12.46,ES,1633810424
8,Khatanga,71.9667,102.5000,16.14,97,100,1.52,RU,1633810424
9,Palmer,42.1584,-72.3287,63.25,67,100,1.99,US,1633810424


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [4]:
gmaps.configure(api_key=g_key)

In [5]:
locations = cities_df[["Lat", "Lng"]]
weight = cities_df["Humidity"]

In [6]:
fig = gmaps.figure(center=(22.0, -1.0), zoom_level=2)
heat_layer = gmaps.heatmap_layer(locations, weights = weight, 
                                 dissipating=False, max_intensity=np.max(weight),
                                 point_radius=3)
fig.add_layer(heat_layer)
#max_intensity=np.max(weight)
#max_intensity=300
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
cities_new_df = cities_df.dropna()
cities_new_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Sakakah,29.9697,40.2064,81.21,16,0,14.09,SA,1633810422
1,Nikolskoye,59.7035,30.7861,53.87,84,100,5.70,RU,1633810343
2,Yellowknife,62.4560,-114.3525,45.41,98,100,1.99,CA,1633810359
3,Kyra,49.5776,111.9753,28.54,81,13,5.41,RU,1633810423
4,Mishan,45.5500,131.8833,59.18,91,100,14.94,CN,1633810423


In [8]:
filtered_cities = cities_new_df.loc[(cities_new_df["Max Temp"] <= 80) & 
                                    (cities_new_df["Max Temp"] >= 70) & 
                                    (cities_new_df["Wind Speed"] < 10) &  
                                    (cities_new_df["Cloudiness"] == 0)]
                                
filtered_cities

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [9]:
hotel_df = pd.DataFrame(filtered_cities)
hotel_df["Hotel Name"] = ""

In [10]:
target_search = "hotels"

target_radius = 5000
target_type = "Business"

params = {
    "keyword": target_search,
    "radius":target_radius,
    "type":target_type,
    "key" : g_key
}
    
for index, row in hotel_df.iterrows():

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    
    hotel_info = requests.get(base_url, params=params)
    hotel_info = hotel_info.json()
    
    
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_info["results"][0]["name"]
        hotel_df.loc[index, "Hotel Address"] = hotel_info["results"][0]["vicinity"]
        hotel_df.loc[index, "Hotel Rating"] = hotel_info["results"][0]["rating"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

In [11]:
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name


In [12]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [13]:
markers = gmaps.marker_layer(locations,
    info_box_content= hotel_info)

fig.add_layer(markers)
fig

Figure(layout=FigureLayout(height='420px'))